# Census Datasets example

*Goal:* demonstrate basic use of the `census_datasets` dataframe.

Each Cell Census contains a top-level dataframe itemizing the datasets contained therein. You can read this into a Pandas DataFrame:

In [9]:
import cell_census
import tiledbsoma as soma

with cell_census.open_soma() as census:
    census_datasets = census["census_info"]["datasets"].read().concat().to_pandas()

    # for convenience, indexing on the soma_joinid which links this to other census data.
    census_datasets = census_datasets.set_index("soma_joinid")

census_datasets

,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,
0,43d4bb39-21af-4d05-b973-4c1fed7b916c,Transcriptional Programming of Normal and Infl...,10.1016/j.celrep.2018.09.006,f512b8b6-369d-4a85-a695-116e0806857f,Skin,f512b8b6-369d-4a85-a695-116e0806857f.h5ad,68036
1,03cdc7f4-bd08-49d0-a395-4487c0e5a168,Emphysema Cell Atlas,,1e5bd3b8-6a0e-4959-8d69-cafed30fe814,immune cells,1e5bd3b8-6a0e-4959-8d69-cafed30fe814.h5ad,35699
2,03cdc7f4-bd08-49d0-a395-4487c0e5a168,Emphysema Cell Atlas,,214bf9eb-93db-48c8-8e3c-9bb22fa3bc63,AT2 cells,214bf9eb-93db-48c8-8e3c-9bb22fa3bc63.h5ad,3662
3,03cdc7f4-bd08-49d0-a395-4487c0e5a168,Emphysema Cell Atlas,,4b6af54a-4a21-46e0-bc8d-673c0561a836,non-immune cells,4b6af54a-4a21-46e0-bc8d-673c0561a836.h5ad,18386
4,0434a9d4-85fd-4554-b8e3-cf6c582bb2fa,Acute COVID-19 cohort across a range of WHO ca...,10.1101/2020.11.20.20227355,fa8605cf-f27e-44af-ac2a-476bee4410d3,PBMCs,fa8605cf-f27e-44af-ac2a-476bee4410d3.h5ad,59506
...,...,...,...,...,...,...,...
473,93eebe82-d8c3-41bc-a906-63b5b5f24a9d,Single-cell proteo-genomic reference maps of t...,10.1038/s41590-021-01059-0,cd4c96bb-ad66-4e83-ba9e-a7df8790eb12,3 healthy young and 3 healthy old bone marrow ...,cd4c96bb-ad66-4e83-ba9e-a7df8790eb12.h5ad,49057
474,93eebe82-d8c3-41bc-a906-63b5b5f24a9d,Single-cell proteo-genomic reference maps of t...,10.1038/s41590-021-01059-0,c05fb583-eb2f-4e3a-8e74-f9bd6414e418,healthy young bone marrow donor,c05fb583-eb2f-4e3a-8e74-f9bd6414e418.h5ad,13165
475,f70ebd97-b3bc-44fe-849d-c18e08fe773d,A transcriptomic atlas of the mouse cerebellum...,10.1101/2020.03.04.976407,e0ed3c55-aff6-4bb7-b6ff-98a2d90b890c,A transcriptomic atlas of the mouse cerebellum,e0ed3c55-aff6-4bb7-b6ff-98a2d90b890c.h5ad,611034


The sum cells across all datasets should match the number of cells across all SOMA experiments (human, mouse).

In [2]:
# Count cells across all experiments
with cell_census.open_soma() as census:
    all_experiments = (
        (organism_name, organism_experiment) for organism_name, organism_experiment in census["census_data"].items()
    )
    experiments_total_cells = 0
    print("Count by experiment:")
    for organism_name, organism_experiment in all_experiments:
        num_cells = len(organism_experiment.obs.read(column_names=["soma_joinid"]).concat().to_pandas())
        print(f"\t{num_cells} cells in {organism_name}")
        experiments_total_cells += num_cells

    print(f"\nFound {experiments_total_cells} cells in all experiments.")

    # Count cells across all datasets
    print(f"Found {census_datasets.dataset_total_cell_count.sum()} cells in all datasets.")

Count by experiment:
	43420131 cells in homo_sapiens
	3922090 cells in mus_musculus

Found 47342221 cells in all experiments.
Found 47342221 cells in all datasets.


Lets pick one dataset to slice out of the census, and turn into an [AnnData](https://anndata.readthedocs.io/en/latest/) in-memory object. This can be used with the [ScanPy](https://scanpy.readthedocs.io/en/stable/) toolchain. You can also save this AnnData locally using the AnnData [`write`](https://anndata.readthedocs.io/en/latest/api.html#writing) API.

In [3]:
census_datasets[census_datasets.dataset_id == "0bd1a1de-3aee-40e0-b2ec-86c7a30c7149"]

,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,
224,0b9d8a04-bb9d-44da-aa27-705bb65b54eb,Tabula Muris Senis,10.1038/s41586-020-2496-1,0bd1a1de-3aee-40e0-b2ec-86c7a30c7149,Bone marrow - A single-cell transcriptomic atl...,0bd1a1de-3aee-40e0-b2ec-86c7a30c7149.h5ad,40220


Create a query on the mouse experiment, "RNA" measurement, for the dataset_id.

In [7]:
with cell_census.open_soma() as census:
    mouse = census["census_data"]["mus_musculus"]
    with mouse.axis_query(
        "RNA",
        obs_query=soma.AxisQuery(value_filter="dataset_id == '0bd1a1de-3aee-40e0-b2ec-86c7a30c7149'"),
    ) as query:
        adata = query.to_anndata("raw")

adata

AnnData object with n_obs × n_vars = 40220 × 52392
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

You can also use the `cell_census.get_h5ad_uri()` API to fetch a URI pointing to the H5AD associated with this `dataset_id`. This is the same H5AD you can download from the CELLxGENE Portal, and may contain additional data-submittor provided information which was not included in the Cell Census.

The "locator" returned by this API will include a `uri` and additional information that may be necessary to use the URI (eg, the S3 region).

You will need to use a download API to fetch this H5AD, such as [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/).

In [8]:
with cell_census.open_soma() as census:
    uri = cell_census.get_source_h5ad_uri("0bd1a1de-3aee-40e0-b2ec-86c7a30c7149")

uri

{'uri': 's3://cellxgene-data-public/cell-census/2023-02-06/h5ads/0bd1a1de-3aee-40e0-b2ec-86c7a30c7149.h5ad',
 's3_region': 'us-west-2'}